### Purpose

Find nudge impact for distributions generated by SIRS disease models

In [ ]:
import time
import os
import json
import numpy as np
import networkx as nx
import pandas as pd
from r0bit import ising
import ising_model

### load samples and networks to distributions

load the text file with the samples and the distribution of the networks

In [ ]:
number_of_networks = 100
number_of_nodes = 100
network_degree = 2
mode = "powerlaw"
file_path = "/home/derkjan/Documents/academics_UVA/master_thesis_code/master_thesis/"
directory = "network_files/"
file_format = "network{}nodes{}degree{}mode_{}.json"
number_of_simulations = 100000
infection_chance = 0.1
directory_out = "networkDiseaseFiles/"
file_format_out = "network{}nodes{}degree{}mode_{}.json"   


In [ ]:
class Person():
    def __init__(self, number, contacts=None):
        self.id = number
        self.contacts = [] if contacts is None else contacts
        
class Network():
    def __init__(self, id_to_persons):
        """
        Parameters:
        ----------
        id_to_persons: a dict
        
        """
        self.id_to_persons = id_to_persons
        self.id_to_index = {personId:count for count, personId in enumerate(sorted(id_to_persons.keys()))}

    def person_id_to_index(self, index):
        self.persons.keys()
        
    def get_person(self, _id):
        return self.id_to_persons[_id]
        
def convert_keys_to_str(dictionary):
    return {str(k):v for k,v in dictionary.items()}




In [ ]:

networks = []
network_samples = []
for network_number in range(number_of_networks):
    filename_in = file_format.format(network_number, number_of_nodes, network_degree, mode)
    with open(file_path+directory+filename_in, 'r') as f:
        persons = json.load(f)

    persons = [convert_keys_to_str(i) for i in persons]
    persons = [Person(person["id"], person["neighbors"]) for person in persons]
    networks.append(Network({person.id: person for person in persons}))

    filename_out = file_format_out.format(network_number, number_of_nodes, network_degree, mode)
    with open(file_path+directory_out+filename_out, 'r') as f:
        network_samples.append(json.load(f))
        

In [ ]:
#np.array(network_samples[1])
print(networks[0].id_to_index)
print(len(network_samples[0][0]))

### Convert the samples to distributions

#### settings for creating the distributions

In [ ]:
number_of_states = 3
max_number_of_neighbors = 7
number_of_nodes = 100
network_degree = 2
file_path = "/home/derkjan/Documents/academics_UVA/master_thesis_code/master_thesis/"
distributions_folder = "SIRS_distributions/"
if not os.path.exists(distributions_folder):
    os.makedirs(distributions_folder)

In [ ]:

def get_number_of_neighbor_to_ids(network, max_number_of_neighbors):
    id_to_number_of_neighbors = {person.id:len(person.contacts) for person in network.id_to_persons.values()}
    number_of_neighbor_to_ids = {i:[] for i in range(1, max_number_of_neighbors+1, 1)}
    for person_id, number_of_neighbors in sorted(id_to_number_of_neighbors.items(), key=lambda x: x[1]):
        if number_of_neighbors > max_number_of_neighbors:
            break

        number_of_neighbor_to_ids[number_of_neighbors].append(person_id)
    
    #print(number_of_neighbor_to_ids)
    return number_of_neighbor_to_ids

def select_nodes(network, max_number_of_neighbors):
    """
    
    Returns: a list of ints representing the ids of the persons chosen
    """
    number_of_neighbor_to_ids = get_number_of_neighbor_to_ids(network, max_number_of_neighbors)
    #print(number_of_neighbor_to_ids.values())
    return [np.random.choice(i) for i in number_of_neighbor_to_ids.values() if i != []]

def convert_samples_to_distribution(samples, number_of_states):
    """

    :param samples: a 2-d nd-array 
    :param nubmer_of_states: a list, the number of states of every var 
    :return: an nd-array
    """
    distribution = np.zeros(tuple(number_of_states))
    number_of_samples = samples.shape[0]
    for sample_number in range(number_of_samples):
        indices = tuple(int(i) for i in samples[sample_number, :])
        distribution[indices] += 1

    return distribution/float(np.sum(distribution))

def save_marginal_and_conditional(distribution, filepath_marginal, filepath_conditional):
    output_label = set([len(distribution.shape) - 1])
    input_labels = set(range(len(distribution.shape) - 1))
    marginal, conditional = ising_model.produce_marginal_and_conditional(
        distribution, input_labels, output_label
    )

    np.save(filename_marginal, marginal)
    np.save(filename_conditional, conditional)



In [ ]:
array = np.array([
    [0, 0, 2],
    [1, 1, 0],
    [2, 0, 2],
    [0, 0, 2],
    [2, 2, 1]
])
dist = convert_samples_to_distribution(array, [3, 3, 3])
print(dist.flatten().shape[0])

In [ ]:
marginal_format = "{}/marginal{}_neighbors{}_network_size{}_network_degree{}_mode_{}.npy"
conditional_format = "{}/conditional{}_neighbors{}_network_size{}_network_degree{}_mode_{}.npy"
for network_number in range(number_of_networks):
    print("network count {}".format(network_number))
    network, network_sample = networks[network_number], network_samples[network_number]
    selected_nodes = select_nodes(network, max_number_of_neighbors)
    
    #select node and its neighbors
    for selected_node in selected_nodes:
        neighbors = network.get_person(selected_node).contacts
        if count == 0:
            print(neighbors)
        
        #so do no use node id but first transfer them to indices
        #IMPORTANT that selected node is appended rather than prepended
        sample_nodes = [network.id_to_index[node_id] for node_id in neighbors+[selected_node]] 
        
        #get samples of node and its neighbors
        samples = np.array(network_sample)[:, sample_nodes]
        distribution = convert_samples_to_distribution(samples, len(sample_nodes)*[3])
        
        filename_marginal = file_path + marginal_format.format(
            distributions_folder, network_number, len(neighbors), 
            number_of_nodes, network_degree, mode
        )
        filename_conditional = file_path + conditional_format.format(
            distributions_folder, network_number, len(neighbors), 
            number_of_nodes, network_degree, mode
        )
        save_marginal_and_conditional(distribution, filename_marginal, filename_conditional)
        


In [ ]:
import numpy as np

a = np.array([
    [1, 2, 3, 4],
    [10, 20, 30, 40]
])
#a[:, [2,1]]
a_list = [2,1]
print(a[:, a_list])

